# Running Dinner

$n$: Gesamtanzahl Teams

$i$: Team

$a$: Team for course 1

$b$: Team for course 2

$c$: Team for course 3

$F$: final location where all teams meet, e.g. a pub

$x_{iabcF}$: Decision variable (DV) Team $i$ is going to team $a$ for course 1, to team $b$ for course 2, to team $c$ for course 3, and finally to the final location $F$

$x_{ik}$: Decision variable Team $i$ is preparing course $k$

$d_{iabcF}$: Total commuting time for team $i$ for route to team $a$, then to team $b$, then to team $c$, and finally to the final location $F$
<br><br>

Every team is exactly 1x hosting and has exactly 1 route:

$\Sigma_{b=1}^n \Sigma_{c=1}^n x_{iibcF} + \Sigma_{a=1}^n \Sigma_{c=1}^n x_{iaicF} + \Sigma_{a=1}^n \Sigma_{b=1}^n x_{iabiF} = 1$ &nbsp; $\forall i$
<br><br>

Every team is paired with the same team max. 1x:

$\Sigma_{i=1}^n x_{iibcF} \leq 1$ &nbsp; $\forall b,c,$ &nbsp; $b \not = c$
<br>

$\Sigma_{i=1}^n x_{iaicF} \leq 1$ &nbsp; $\forall a,c,$ &nbsp; $a \not = c$
<br>

$\Sigma_{i=1}^n x_{iabiF} \leq 1$ &nbsp; $\forall a,b,$ &nbsp; $a \not = b$
<br><br>

For course 1, exactly 3 teams are at the same place:

$0 \leq \Sigma_{i=1}^n \Sigma_{b=1}^n \Sigma_{c=1}^n x_{iabcF} \leq 3 * x_{a1}$ &nbsp; $\forall a$
<br><br>

For course 2, exactly 3 teams are at the same place:

$0 \leq \Sigma_{i=1}^n \Sigma_{a=1}^n \Sigma_{c=1}^n x_{iabcF} \leq 3 * x_{b2}$ &nbsp; $\forall b$
<br><br>

For course 3, exactly 3 teams are at the same place:

$0 \leq \Sigma_{i=1}^n \Sigma_{a=1}^n \Sigma_{b=1}^n x_{iabcF} \leq 3 * x_{c3}$ &nbsp; $\forall c$
<br><br>

Every team is hosting exactly 1 course:

$\Sigma_{k=1}^3 x_{ik} = 1$ &nbsp; $\forall i$
<br><br>

Every course is hosted by exactly $n/3$ teams:

$\Sigma_{i=1}^n x_{ik} = n/3$ &nbsp; $\forall k$
<br><br>

A team must not be paired with themself and has to be part of their own route:

$x_{iabcF} = 0$ &nbsp; $for$ &nbsp; $a == b \lor a == c \lor b == c \lor (i \not = a \land i \not = b \land i \not = c)$
<br><br>

Decision variable is either $0$ or $1$:

$0 \leq x_{iabcF} \leq 1$ &nbsp; $\forall i, a, b, c$
<br>
$0 \leq x_{ik} \leq 1$ &nbsp; $\forall i, k$
<br><br>

Minimise the total commuting time for all teams:
$\Sigma_{i=1}^n \Sigma_{a=1}^n \Sigma_{b=1}^n \Sigma_{c=1}^n x_{iabcF} * d_{iabcF} = min$

In [1]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import logging
import os
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, PULP_CBC_CMD

In [2]:
model = LpProblem('Running_Dinner', LpMinimize)

In [3]:
# total number of teams n
n = 15

In [4]:
# distance matrix
distance_matrix = np.array([[0, 51, 46, 33, 45, 57, 23, 51, 45, 46, 48, 53, 45, 47, 54, 45],
                           [56, 0, 21, 38, 9, 47, 32, 14, 15, 25, 22, 31, 9, 30, 33, 26],
                           [44, 20, 0, 25, 12, 36, 17, 16, 13, 38, 22, 26, 12, 31, 46, 39],
                           [30, 37, 26, 0, 35, 41, 11, 31, 28, 28, 32, 41, 33, 32, 36, 26],
                           [45, 9, 14, 35, 0, 40, 26, 15, 10, 31, 15, 24, 1, 32, 39, 32],
                           [57, 45, 33, 40, 40, 0, 38, 38, 35, 34, 37, 52, 40, 20, 35, 32],
                           [23, 34, 18, 10, 25, 41, 0, 23, 20, 26, 22, 32, 25, 33, 30, 27],
                           [50, 14, 16, 30, 15, 41, 22, 0, 15, 34, 27, 27, 15, 39, 42, 35],
                           [44, 14, 13, 28, 10, 39, 20, 15, 0, 32, 25, 26, 10, 30, 40, 33],
                           [45, 25, 37, 33, 33, 32, 25, 34, 31, 0, 23, 43, 31, 30, 13, 9],
                           [46, 22, 22, 32, 15, 35, 24, 30, 25, 26, 0, 33, 17, 25, 34, 27],
                           [55, 33, 26, 39, 23, 49, 32, 27, 26, 42, 33, 0, 28, 43, 50, 43],
                           [45, 9, 14, 32, 1, 40, 26, 15, 10, 31, 17, 23, 0, 32, 39, 32],
                           [47, 34, 30, 31, 37, 20, 33, 35, 32, 31, 27, 42, 37, 0, 39, 32],
                           [53, 33, 45, 35, 40, 33, 33, 42, 39, 13, 31, 51, 41, 39, 0, 17]])
#print(distance_matrix)

In [5]:
total_distance_matrix = np.full((n, n, n, n, 1), 999999999)
list_letters = [chr(i) for i in range(ord('a'),ord('a')+n)]
variable_names = list()

for i in range(n):
    for a in range(n):
        for b in range(n):
            for c in range(n):
                if (a != b) & (a != c) & (b != c) & (distance_matrix[i][a] >= 0) & (distance_matrix[i][b] >= 0) & (distance_matrix[i][c] >= 0):
                    total_distance_matrix[i][a][b][c][0] = distance_matrix[i][a] + distance_matrix[a][b] + distance_matrix[b][c] + distance_matrix[c][n]
                variable_names.append(f'{list_letters[i]}{list_letters[a]}{list_letters[b]}{list_letters[c]}{0}')

Decision variable is either $0$ or $1$:

$0 \leq x_{iabcF} \leq 1$ &nbsp; $\forall i, a, b, c$

In [6]:
dv_variables = LpVariable.matrix('x', variable_names, cat='Integer', lowBound=0, upBound=1)
allocation = np.array(dv_variables).reshape(n, n, n, n, 1)

In [7]:
iv_variable_names = list()
for i in range(n):
    for g in range(3):
        iv_variable_names.append(f'{list_letters[i]}{g+1}')

Decision variable is either $0$ or $1$:

$0 \leq x_{ik} \leq 1$ &nbsp; $\forall i, k$

In [8]:
iv_variables = LpVariable.matrix('x', iv_variable_names, cat='Integer', lowBound=0, upBound=1)
iv_allocation = np.array(iv_variables).reshape(n, 3)

Minimise the total commuting time for all teams:

$\Sigma_{i=1}^n \Sigma_{a=1}^n \Sigma_{b=1}^n \Sigma_{c=1}^n x_{iabcF} * d_{iabcF} = min$

In [9]:
objective_function = lpSum(allocation * total_distance_matrix)

model += objective_function

A team must not be paired with themself and has to be part of their own route:

$x_{iabcF} = 0$ &nbsp; $for$ &nbsp; $a == b \lor a == c \lor b == c \lor (i \not = a \land i \not = b \land i \not = c)$

In [10]:
model += lpSum(allocation[i][a][b][c][0] for i in range(n) for a in range(n) for b in range(n) for c in range(n) if total_distance_matrix[i][a][b][c][0] >= 999999999) == 0, 'Keine negative Distanz'

Every team is exactly 1x hosting and has exactly 1 route:

$\Sigma_{b=1}^n \Sigma_{c=1}^n x_{iibcF} + \Sigma_{a=1}^n \Sigma_{c=1}^n x_{iaicF} + \Sigma_{a=1}^n \Sigma_{b=1}^n x_{iabiF} = 1$ &nbsp; $\forall i$
<br><br>

In [11]:
for i in range(n):
    #print(lpSum(allocation[i][i][b][c][0] for b in range(n) for c in range(n) if (i != b) & (i != c) & (b != c)) + \
    #      lpSum(allocation[i][a][i][c][0] for a in range(n) for c in range(n) if (i != a) & (i != c) & (a != c)) + \
    #      lpSum(allocation[i][a][b][i][0] for a in range(n) for b in range(n) if (i != a) & (i != b) & (a != b)) == 1)
    model += lpSum(allocation[i][i][b][c][0] for b in range(n) for c in range(n) if (i != b) & (i != c) & (b != c)) + \
          lpSum(allocation[i][a][i][c][0] for a in range(n) for c in range(n) if (i != a) & (i != c) & (a != c)) + \
          lpSum(allocation[i][a][b][i][0] for a in range(n) for b in range(n) if (i != a) & (i != b) & (a != b)) == 1, 'Eine Strecke für Team ' + str(i)

Every team is exactly 1x hosting and has exactly 1 route:

$\Sigma_{i=1}^n x_{iibcF} \leq 1$ &nbsp; $\forall b,c,$ &nbsp; $b \not = c$
<br>

$\Sigma_{i=1}^n x_{iaicF} \leq 1$ &nbsp; $\forall a,c,$ &nbsp; $a \not = c$
<br>

$\Sigma_{i=1}^n x_{iabiF} \leq 1$ &nbsp; $\forall a,b,$ &nbsp; $a \not = b$

In [12]:
for a in range(n):
    for b in range(n):
        if (a != b):
            #print(lpSum(allocation[i][a][b][c] for i in range(n) for c in range(n) if (a != c) & (b != c) & (a != b)) <= 1)
            
            model += lpSum(allocation[i][a][b][c] for i in range(n) for c in range(n) if (a != c) & (b != c) & (a != b)) <= 1, f'Maximal 1 Team ist sowohl bei Team {a} im 1. Gang und bei Team {b} im 2. Gang'

In [13]:
for a in range(n):
    for c in range(n):
        if (a != c):
            #print(lpSum(allocation[i][a][b][c] for i in range(n) for b in range(n) if (a != c) & (b != c) & (a != b)) <= 1)
            
            model += lpSum(allocation[i][a][b][c] for i in range(n) for b in range(n) if (a != c) & (b != c) & (a != b)) <= 1, f'Maximal 1 Team ist sowohl bei Team {a} im 1. Gang und bei Team {c} im 3. Gang'

In [14]:
for b in range(n):
    for c in range(n):
        if (b != c):
            #print(lpSum(allocation[i][a][b][c] for i in range(n) for a in range(n) if (a != c) & (b != c) & (a != b)) <= 1)
            
            model += lpSum(allocation[i][a][b][c] for i in range(n) for a in range(n) if (a != c) & (b != c) & (a != b)) <= 1, f'Maximal 1 Team ist sowohl bei Team {b} im 2. Gang und bei Team {c} im 3. Gang'

For course 1, exactly 3 teams are at the same place:

$0 \leq \Sigma_{i=1}^n \Sigma_{b=1}^n \Sigma_{c=1}^n x_{iabcF} \leq 3 * x_{a1}$ &nbsp; $\forall a$
<br><br>

In [15]:
for a in range(n):
    #print(lpSum(allocation[i][a][b][c][0] \
    #            for i in range(n) for b in range(n) for c in range(n) \
    #            if (a != b) & (a != c) & (b != c) & ((i == a) or (i == b) or (i == c))) >= 0)
    model += lpSum(allocation[i][a][b][c][0] \
                for i in range(n) for b in range(n) for c in range(n) \
                if (a != b) & (a != c) & (b != c) & ((i == a) or (i == b) or (i == c))) >= 0, f'Min 3 Teams bei Team {a} im 1. Gang'
    
    #print(lpSum(allocation[i][a][b][c][0] \
    #            for i in range(n) for b in range(n) for c in range(n) \
    #            if (a != b) & (a != c) & (b != c) & ((i == a) or (i == b) or (i == c))) \
    #      - 3 * iv_allocation[a][0] <= 0)
    model += lpSum(allocation[i][a][b][c][0] \
                for i in range(n) for b in range(n) for c in range(n) \
                if (a != b) & (a != c) & (b != c) & ((i == a) or (i == b) or (i == c))) \
          - 3 * iv_allocation[a][0] <= 0, f'Max 3 Teams bei Team {a} im 1. Gang'

For course 2, exactly 3 teams are at the same place:

$0 \leq \Sigma_{i=1}^n \Sigma_{a=1}^n \Sigma_{c=1}^n x_{iabcF} \leq 3 * x_{b2}$ &nbsp; $\forall b$
<br><br>

In [16]:
for b in range(n):
    #print(lpSum(allocation[i][a][b][c][0] \
    #            for i in range(n) for a in range(n) for c in range(n) \
    #            if (a != b) & (a != c) & (b != c) & ((i == a) or (i == b) or (i == c))) >= 0)
    model += lpSum(allocation[i][a][b][c][0] \
                for i in range(n) for a in range(n) for c in range(n) \
                if (a != b) & (a != c) & (b != c) & ((i == a) or (i == b) or (i == c))) >= 0, f'Min 3 Teams bei Team {b} im 2. Gang'
    
    #print(lpSum(allocation[i][a][b][c][0] \
    #           for i in range(n) for a in range(n) for c in range(n) \
    #            if (a != b) & (a != c) & (b != c) & ((i == a) or (i == b) or (i == c))) \
    #      - 3 * iv_allocation[b][1] <= 0)
    model += lpSum(allocation[i][a][b][c][0] \
                for i in range(n) for a in range(n) for c in range(n) \
                if (a != b) & (a != c) & (b != c) & ((i == a) or (i == b) or (i == c))) \
          - 3 * iv_allocation[b][1] <= 0, f'Max 3 Teams bei Team {b} im 2. Gang'

For course 3, exactly 3 teams are at the same place:

$0 \leq \Sigma_{i=1}^n \Sigma_{a=1}^n \Sigma_{b=1}^n x_{iabcF} \leq 3 * x_{c3}$ &nbsp; $\forall c$
<br><br>

In [17]:
for c in range(n):
    #print(lpSum(allocation[i][a][b][c][0] \
    #            for i in range(n) for a in range(n) for b in range(n) \
    #            if (a != b) & (a != c) & (b != c) & ((i == a) or (i == b) or (i == c))) >= 0)
    model += lpSum(allocation[i][a][b][c][0] \
                for i in range(n) for a in range(n) for b in range(n) \
                if (a != b) & (a != c) & (b != c) & ((i == a) or (i == b) or (i == c))) >= 0, f'Min 3 Teams bei Team {c} im 3. Gang'
    
    #print(lpSum(allocation[i][a][b][c][0] \
    #            for i in range(n) for a in range(n) for b in range(n) \
    #            if (a != b) & (a != c) & (b != c) & ((i == a) or (i == b) or (i == c))) \
    #      - 3 * iv_allocation[c][2] <= 0)
    model += lpSum(allocation[i][a][b][c][0] \
                for i in range(n) for a in range(n) for b in range(n) \
                if (a != b) & (a != c) & (b != c) & ((i == a) or (i == b) or (i == c))) \
          - 3 * iv_allocation[c][2] <= 0, f'Max 3 Teams bei Team {c} im 3. Gang'

Every team is hosting exactly 1 course:

$\Sigma_{k=1}^3 x_{ik} = 1$ &nbsp; $\forall i$

In [18]:
for i in range(n):
    #print(lpSum(iv_allocation[i][k] for k in range(3)) == 1)
    model += lpSum(iv_allocation[i][k] for k in range(3)) == 1, f'Team {i} hat genau 1 Gang'

Every course is hosted by exactly $n/3$ teams:

$\Sigma_{i=1}^n x_{ik} = 3$ &nbsp; $\forall k$

In [19]:
for k in range(3):
    #print(lpSum(iv_allocation[i][k] for i in range(n)) == n/3)
    model += lpSum(iv_allocation[i][k] for i in range(n)) == n/3, f'Gang {k} wird von genau {n/3} Teams gekocht'

In [21]:
def get_dv_variable_index(n,i,a,b,c):
    print(dv_variables[(i*pow(n,3)) + (a*pow(n,2)) + (b*pow(n,1)) + (c*pow(n,0))])
    print((i*pow(n,3)) + (a*pow(n,2)) + (b*pow(n,1)) + (c*pow(n,0)))

In [42]:
get_dv_variable_index(n, 14, 14, 2, 13)

x_oocn0
50443


In [23]:
#x[('O', 'P', 'Q', 'R')].setInitialValue(1)
#x[('K', 'N', 'O', 'R')].setInitialValue(0)

dv_variables[97].setInitialValue(1) #1
dv_variables[3672].setInitialValue(1) #2
dv_variables[7012].setInitialValue(1) #3
dv_variables[10902].setInitialValue(1) #4
dv_variables[14246].setInitialValue(1) #5
dv_variables[18041].setInitialValue(1) #6
dv_variables[23499].setInitialValue(1) #7
dv_variables[24427].setInitialValue(1) #8
dv_variables[27129].setInitialValue(1) #9
dv_variables[30759].setInitialValue(1) #10
dv_variables[33913].setInitialValue(1) #11
dv_variables[40436].setInitialValue(1) #12
dv_variables[41757].setInitialValue(1) #13
dv_variables[45073].setInitialValue(1) #14
dv_variables[50443].setInitialValue(1) #15

True

In [24]:
iv_variables[0*3 + 0].setInitialValue(1)
iv_variables[1*3 + 0].setInitialValue(1)
iv_variables[2*3 + 1].setInitialValue(1)
iv_variables[3*3 + 0].setInitialValue(1)
iv_variables[4*3 + 1].setInitialValue(1)
iv_variables[5*3 + 0].setInitialValue(1)
iv_variables[6*3 + 1].setInitialValue(1)
iv_variables[7*3 + 2].setInitialValue(1)
iv_variables[8*3 + 1].setInitialValue(1)
iv_variables[9*3 + 2].setInitialValue(1)
iv_variables[10*3 + 1].setInitialValue(1)
iv_variables[11*3 + 2].setInitialValue(1)
iv_variables[12*3 + 2].setInitialValue(1)
iv_variables[13*3 + 2].setInitialValue(1)
iv_variables[14*3 + 0].setInitialValue(1)

True

In [25]:
time_limit_in_seconds = 60*60*2
model.solve(PULP_CBC_CMD(warmStart=True, timeLimit=time_limit_in_seconds))
status = LpStatus[model.status]
print(status)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8b/z587n0ws2lg0_hvpyjtpn3vr0000gp/T/0cab028dc55446548d667b7744523535-pulp.mps mips /var/folders/8b/z587n0ws2lg0_hvpyjtpn3vr0000gp/T/0cab028dc55446548d667b7744523535-pulp.mst sec 7200 timeMode elapsed branch printingOptions all solution /var/folders/8b/z587n0ws2lg0_hvpyjtpn3vr0000gp/T/0cab028dc55446548d667b7744523535-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 760 COLUMNS
At line 343080 RHS
At line 343836 BOUNDS
At line 394507 ENDATA
Problem MODEL has 755 rows, 50670 columns and 190354 elements
Coin0008I MODEL read with 0 errors
opening mipstart file /var/folders/8b/z587n0ws2lg0_hvpyjtpn3vr0000gp/T/0cab028dc55446548d667b7744523535-pulp.mst.
MIPStart values read for 50670 variables.
seconds was changed from 1e+100 to 7200
Option for timeMode 

In [26]:
print('Total Distance:', model.objective.value())

Total Distance: 1310.0


In [23]:
print('Total Distance:', model.objective.value())

Total Distance: 1275.0


In [27]:
for v in model.variables():
    try:
        if (v.value() > 0) & (len(v.name) < 5):
            print(v.name, '=', v.value())
    except:
        pass
        #print('error couldnt find value')

x_a1 = 1.0
x_b3 = 1.0
x_c2 = 1.0
x_d3 = 1.0
x_e2 = 1.0
x_f1 = 1.0
x_g2 = 1.0
x_h2 = 1.0
x_i3 = 1.0
x_j3 = 1.0
x_k3 = 1.0
x_l1 = 1.0
x_m1 = 1.0
x_n2 = 1.0
x_o1 = 1.0
